# The second part of the project:

### 1. Store the inverted index and document magnitude created in first part, in a relational database (SQLite) 
### 2. User input Key words search from the documents - 
Searches for the query text (word or sentence) input by the user in the SQLite tables containing the inverted    index and document magnitude. 
### 3. Ranks the documents containing the query text in decreasing order of cosine similarity. 
The cosine similarity between the documents and the query text is calculated and the most similar documents appears at the top. 


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
import sqlite3
con = sqlite3.connect('keyword_search_project.db')
cur = con.cursor() #creating cursor for db operation

#creating table

cur.execute('''DROP TABLE IF EXISTS postings ''')
cur.execute('''DROP TABLE IF EXISTS docmag ''')

cur.execute('''CREATE TABLE IF NOT EXISTS postings ( word VARCHAR(100) PRIMARY KEY, postinglist_freq_tfidf TEXT)''')
cur.execute('''CREATE TABLE IF NOT EXISTS docmag ( docid INT PRIMARY KEY,maxf INT,mag FLOAT)''')

cur.close()#closing the cursor
#commiting changes
con.commit()

#closing connection
con.close()

In [21]:
# for insering into db
con = sqlite3.connect('keyword_search_project.db')
cur = con.cursor()

import re #regular expression

# Insering the document magnitude in to the table docmag

docmag_arr = []
with open('/content/drive/My Drive/doc_mag.txt') as my_file:
  docmag_arr = my_file.readlines()
for ele in docmag_arr:
  ele = re.findall(r'\d+\.?\d*', ele) #finding elements using re
  doc_id = int(ele[0])
  max_fq = int(ele[1])
  mag = float(ele[2])
  cur.execute("INSERT INTO DOCMAG VALUES (?,?,?)",(doc_id,max_fq,mag))

# Insering the inverted index in to the table in postings
posting_arr = []
with open('/content/drive/My Drive/inv_idx.txt') as my_file:
  posting_arr = my_file.readlines()
for ele in posting_arr:
  ele = re.findall(r'\'(.*)\'[^\[]*(\[.*\])', ele)
  word = ele[0][0]
  posting_tf_idf = ele[0][1]
  cur.execute("INSERT INTO postings VALUES (?,?)",(word,posting_tf_idf))
  
cur.close()
con.commit()
con.close()


In [22]:
from bs4 import BeautifulSoup
from collections import Counter
import re
import string
import math
from string import digits
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
con = sqlite3.connect('keyword_search_project.db')
cur = con.cursor()

# Function to search the text entered (word or sentence)

def searchFortext(event):

  # Cleaning the query text input (word or sentence) by the user in the key word search. 
  search_word_text_lower = textbox.value.lower() #converting to lower case
  search_word_text_notags = BeautifulSoup(search_word_text_lower).get_text() #removing html tags
  remove_digits = str.maketrans('','',digits) 
  search_word_text_nodigits = search_word_text_notags.translate(remove_digits) #removing numbers 
  search_word_text_nosplchar = re.sub('[^A-Za-z0-9]+',' ',search_word_text_nodigits) #remove special characters
  search_word_text_nopunct = ''
  for char in search_word_text_nosplchar:
    if char not in punctuations:
      search_word_text_nopunct = search_word_text_nopunct + char #remove punctuations

  search_word_text_tokenize = word_tokenize(search_word_text_nopunct) #tokenized text for getting this us an individual text
  
  search_word_text_nostopwords = [word for word in search_word_text_tokenize if word not in stopwords.words('english')] #removing stopwords
  counter = Counter(search_word_text_nostopwords) # counting the number of words

  maxfrequency_word = counter.most_common(1)[0][1] # finding maximum number of times a word is getting repeated in the document
  word_dictionary = dict() 
  for obj in counter: 
    word_tf = counter[obj]/maxfrequency_word
    word_tfidf = word_tf * (1/len(counter))
    word_dictionary[obj] = word_tfidf
  search_query = "','".join(search_word_text_nostopwords) #we are creating comma seperated string for our select query
  sql = "SELECT * FROM postings WHERE word IN ('?')"
  sql = sql.replace("?",search_query,1) 
  cur.execute(sql)
  result = cur.fetchall()

  process(result,word_dictionary)

def process(result,word_dictionary):
  docid_dictionary = dict() # To store tfidf of words with respect to document id
  docmag_dictionary = dict() # To store indiviual docids and its magnitude
  for ele in result:
    element = re.findall(r'\((.*?),.*?[+-]?([0-9]*[.]?[0-9]+)\)', ele[1])# doc id & postings tfidf
    for e in element :
      docid_dictionary[e[0]] = e[1]
    
  for docid in docid_dictionary:
    docmag_sql = "SELECT * FROM docmag WHERE docid = ?"
    cur.execute(docmag_sql,docid)
    result_docmag = cur.fetchone()
    docmag_dictionary[result_docmag[0]] = result_docmag[2] # docid and mag

  # COSINE SIMILARITY 

  cosinesimilarity_dictionary = dict()
    
  # Calculating the dot product of the query text (word or sentence) with each document
  for docid in docid_dictionary:
    dotproduct = 0
    for word in word_dictionary:
      mul_tfidf = float(word_dictionary[word])*float(docid_dictionary[docid]) # multiplying word tfidf * doc tf_idf
      dotproduct = dotproduct + mul_tfidf

    # Calculating the modulus of the query text
    querymodulus = 0
    for word in word_dictionary:
      word_tfidf = float(word_dictionary[word])*float(word_dictionary[word])
      querymodulus = querymodulus + word_tfidf
        
    querymodulus_sqrt = math.sqrt(querymodulus)
    
    # Calculating the modulus of the document
    docmuentmag_modulus = math.sqrt(docmag_dictionary[int(docid)])# we have the mag from docmag dict
    
    # Calculating the cosine similarities of all the documents with the query text, and storing them in a dictionary 
    cosinesimilarity_dictionary[docid] = dotproduct/(querymodulus_sqrt*docmuentmag_modulus)
  
  # Sorting the cosine similarities
  sorted_cosine = sorted(cosinesimilarity_dictionary.items(), key=lambda x: x[1], reverse=False) 
  for rows in sorted_cosine:
    print(rows[0])



In [23]:
# User Interface for key word search.
# Clicking the 'Search' button triggers the query search and go to function 'searchFortext'

import ipywidgets as widgets
textbox = widgets.Text(
    value='',
    placeholder='Enter word to Search',
    disabled=False
)

button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
result = widgets.Output()
button.on_click(searchFortext)
widgets.VBox([textbox,button,result])

1
4
5
2
3
